In [ ]:
import pandas as pd

file_path = "demo.csv"
df = pd.read_csv(file_path, encoding="cp1258")
df_small = df[["diem", "ttnv"]].copy()


df_small["diem"] = df_small["diem"].astype(str).str.replace(",", ".")
df_small["diem"] = df_small["diem"].astype(float)

def convert_pref(x):
    if pd.isna(x):
        return None
    x = str(x).strip()
    if x.endswith(('1','2','3','4','5','6','7','8','9')):
        num = ''.join([c for c in x if c.isdigit()])
        # return int(num) if num.isdigit() else None
        return num
    return None
df_small["ttnv"] = df_small["ttnv"].apply(convert_pref)

output_file = "demo_score_seq.csv"
df_small.to_csv(output_file, index=False, encoding="utf-8-sig")

print("✅ Đã xử lý xong, lưu file:", output_file)
print(df_small)


In [12]:
# Câu 4: Kiểm định giả thuyết tỷ lệ p = 0.8 (mức ý nghĩa 5%)
import pandas as pd

file_path = "demo_score_seq.csv"
encodings = ["utf-8-sig", "cp1258", "latin1", "utf-8"]
df = None
for enc in encodings:
    try:
        df = pd.read_csv(file_path, encoding=enc)
        print(f"Read file with encoding: {enc}")
        break
    except Exception as e:
        continue
if df is None:
    raise RuntimeError("Không đọc được file demo_score_seq.csv. Kiểm tra đường dẫn / encoding.")
if "diem" not in df.columns:
    raise RuntimeError("Không tìm thấy cột 'diem' trong file.")
if "ttnv" not in df.columns:
    print("Chú ý: không tìm thấy cột 'ttnv' - sẽ tiếp tục nhưng chỉ dùng 'diem' để tính tỷ lệ >=27")

Read file with encoding: utf-8-sig


In [13]:
df_tren27 = df[df["diem"] >= 27] # type: ignore
n = len(df_tren27)
so_luong_do = (df_tren27['ttnv'] == 1).sum()

ty_le_mau = so_luong_do / n if n> 0 else 0

print(f"Tổng số sinh viên có điểm >= 27 là: {n}")
print(f"Số sinh viên đỗ NV1 trong số đó là : {so_luong_do}")
print(f"Tỷ lệ mẫu quan sát được: {ty_le_mau:.2%}")

Tổng số sinh viên có điểm >= 27 là: 16
Số sinh viên đỗ NV1 trong số đó là : 12
Tỷ lệ mẫu quan sát được: 75.00%


In [14]:
import pandas as pd
import numpy as np
from scipy.stats import norm

alpha = 0.05

# --- Bước 3: Ước lượng Kỳ vọng bằng Khoảng tin cậy ---
# Công thức tính Khoảng tin cậy (CI) cho tỷ lệ:
# CI = p_hat ± Z_(alpha/2) * sqrt(p_hat * (1 - p_hat) / n)
# Với alpha = 0.05, Z_(alpha/2) là giá trị Z cho phân vị 0.975 (norm.ppf(0.975)), xấp xỉ 1.96

z_critical = norm.ppf(1 - alpha / 2)

sai_so_chuan = np.sqrt(ty_le_mau * (1 - ty_le_mau) / n)
sai_so_duong_cong = z_critical * sai_so_chuan

khoang_tin_cay_lower = ty_le_mau - sai_so_duong_cong
khoang_tin_cay_upper = ty_le_mau + sai_so_duong_cong

print("\n--- Ước lượng Khoảng tin cậy ---")
print(f"Giá trị Z-tới hạn: {z_critical:.4f}")
print(f"Khoảng tin cậy {int((1-alpha)*100)}%: [{khoang_tin_cay_lower:.4f}, {khoang_tin_cay_upper:.4f}]")

# --- Bước 4: Kiểm định Giả thuyết với Mức ý nghĩa 5% ---
p_gia_thuyet = 0.80

print("\n--- Kiểm định Giả thuyết ---")
if khoang_tin_cay_lower <= p_gia_thuyet <= khoang_tin_cay_upper:
    print(f"Tỷ lệ giả thuyết ({p_gia_thuyet}) NẰM TRONG khoảng tin cậy.")
    print("Kết luận: CHƯA CÓ ĐỦ BẰNG CHỨNG để bác bỏ giả thuyết ban đầu ở mức ý nghĩa 5%.")
else:
    print(f"Tỷ lệ giả thuyết ({p_gia_thuyet}) NẰM NGOÀI khoảng tin cậy.")
    print("Kết luận: CÓ ĐỦ BẰNG CHỨNG để bác bỏ giả thuyết ban đầu ở mức ý nghĩa 5%.")



--- Ước lượng Khoảng tin cậy ---
Giá trị Z-tới hạn: 1.9600
Khoảng tin cậy 95%: [0.5378, 0.9622]

--- Kiểm định Giả thuyết ---
Tỷ lệ giả thuyết (0.8) NẰM TRONG khoảng tin cậy.
Kết luận: CHƯA CÓ ĐỦ BẰNG CHỨNG để bác bỏ giả thuyết ban đầu ở mức ý nghĩa 5%.
